# 1.进程之间的内存空间是共享的还是隔离的？

In [4]:
%%writefile  temp_multi.py
from multiprocessing import Process

n=100

def work():
    global n
    n=0
    print("子进程n：{}".format(n))

if __name__ == "__main__":
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"

    p=Process(target=work,)
    p.start()
    p.join()

    print("主进程n：{}".format(n))

Overwriting temp_multi.py


# 2.基于多进程实现的并发的套接字通信

## 服务端

In [1]:
%%writefile  server.py
from socket import *
from multiprocessing import Process

def talk(conn):
    while True:
        try:
            data=conn.recv(1024)
            if not data:
                break
            conn.send(data.upper())
        except ConnectionResetError:
            break
    conn.close()


def server(ip,port):
    server = socket(AF_INET, SOCK_STREAM)
    server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    server.bind((ip,port))

    server.listen(5)
    while True:
        conn,add=server.accept()
        p=Process(target=talk,args=(conn,))
        p.start()

    server.close()    


if __name__ == "__main__":
    server('127.0.0.1', 8080)

Writing server.py


In [2]:
%%writefile  client.py
from socket import *

client=socket(AF_INET, SOCK_STREAM)
client.connect(("127.0.0.1", 8080))

while True:
    msg=input('>>: '.strip())
    if not msg:continue

    client.send(msg.encode('utf-8'))
    data=client.recv(1024)
    print(data.decode("utf-8"))

Writing client.py


# 3.进程按顺序执行

In [1]:
%%writefile  temp_multi.py
# 查看子进程的父进程id
from multiprocessing import Process
import time
import random

def task(n):
    time.sleep(random.randint(1, 3))
    print("--------->{}".format(n))
    

if __name__ == "__main__":
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"

    start_time = time.time()
    p1=Process(target=task,args=(1,))
    p2=Process(target=task,args=(2,))
    p3=Process(target=task,args=(3,))
    p_l=[p1,p2,p3]
    
    for p in p_l:
        p.start() # 仅仅只是给操作系统发送了一个信号
        p.join()


    print('--------->4')

Overwriting temp_multi.py
